Piping a RunnableParallel with Other Runnables

In [2]:
%load_ext dotenv

In [3]:
%dotenv

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [5]:
chat_template_books = ChatPromptTemplate.from_template('''
Suggest three of the best intermediate-level {programming language} books.
Answer only by listing the books.
''')


chat_template_projects = ChatPromptTemplate.from_template('''
Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
Answer only by listing the projects.
''')

chat_template_time = ChatPromptTemplate.from_template(
    '''
    I'm an intermediate level programmer. 

    Consider the following literature:
    {books}

    Also, consider the following projrcts: 
    {projects}

    Roughly how much time would it take me to complete the literature and the projects? 
    '''
)

In [6]:
chat = ChatOpenAI(
    model_name = 'gpt-4',
    model_kwargs = {'seed':365},
    temperature = 0,
    max_tokens = 500
)

/opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3519: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [7]:
string_parser = StrOutputParser()

In [8]:
chain_books = chat_template_books | chat | string_parser

chain_projects = chat_template_projects | chat | string_parser

In [9]:
chain_parallel = RunnableParallel({'books': chain_books, 'projects': chain_projects})

In [10]:
chain_parallel.invoke({'programming language': 'Python'})

{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Web Scraper using BeautifulSoup\n2. Developing a simple Machine Learning model using Scikit-learn\n3. Creating a GUI application using Tkinter'}

In [14]:
chain_time1 = (RunnableParallel({'books': chain_books, 'projects': chain_projects}) 
             | chat_template_time | chat | string_parser)

In [15]:
chain_time2 = ({'books': chain_books, 'projects': chain_projects}
             | chat_template_time | chat | string_parser)

In [16]:
print(chain_time2.invoke({'programming language': 'Python'}))

The time it takes to complete the literature and the projects can vary greatly depending on several factors such as your reading speed, comprehension level, the complexity of the projects, and the amount of time you can dedicate each day. 

However, as a rough estimate:

1. "Fluent Python: Clear, Concise, and Effective Programming" - This book is around 800 pages. If you read and practice for about 2 hours a day, it might take you around 2-3 months to complete.

2. "Python Cookbook: Recipes for Mastering Python 3" - This book is around 700 pages. Again, if you read and practice for about 2 hours a day, it might take you around 2-3 months to complete.

3. "Effective Python: 90 Specific Ways to Write Better Python" - This book is around 230 pages. If you read and practice for about 2 hours a day, it might take you around 3-4 weeks to complete.

For the projects:

1. Building a Web Scraper using BeautifulSoup - If you work on it for about 2-3 hours a day, it might take you around 1-2 week

In [17]:
chain_time2.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [ ]:
## using the %%time, compared the time it takes the chains individualy and in parallel. of course in parallel takes less walltime